In [ ]:
# ==============================================================
# 🏋️‍♂️ AI Sports Coach - Colab Test Notebook (Extended)
# ==============================================================
# This notebook tests your posture detection system using:
#  - Webcam frames or static images
#  - Uploaded videos (.mp4)
#  - Real-time posture risk scoring and feedback overlays
# ==============================================================

# --- STEP 1: Install dependencies ---
!pip install mediapipe opencv-python-headless numpy tqdm

# --- STEP 2: Import modules ---
import cv2
import numpy as np
import mediapipe as mp
from tqdm import tqdm
from google.colab import files
from IPython.display import HTML
from base64 import b64encode

# --- STEP 3: Recreate core logic for standalone testing ---
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils


def calculate_angle(a, b, c):
    """Calculate angle between 3 points (in degrees)."""
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba, bc = a - b, c - b
    cos = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-6)
    return np.degrees(np.arccos(np.clip(cos, -1.0, 1.0)))


def detect_pose_angles(frame, pose):
    """Run pose estimation on one frame."""
    h, w = frame.shape[:2]
    results = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    if not results.pose_landmarks:
        return frame, None

    lm = results.pose_landmarks.landmark
    left_shoulder = (lm[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w,
                     lm[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h)
    right_shoulder = (lm[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w,
                      lm[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h)
    left_ear = (lm[mp_pose.PoseLandmark.LEFT_EAR].x * w,
                lm[mp_pose.PoseLandmark.LEFT_EAR].y * h)

    shoulder_angle = calculate_angle(left_shoulder, right_shoulder, (right_shoulder[0], 0))
    neck_angle = calculate_angle(left_ear, left_shoulder, (left_shoulder[0], 0))

    angles = {"shoulder_angle": shoulder_angle, "neck_angle": neck_angle}

    annotated = frame.copy()
    mp_drawing.draw_landmarks(annotated, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    return annotated, angles


def assess_posture_risk(angles):
    """Simple heuristic risk estimation."""
    shoulder_angle = angles.get("shoulder_angle", 0)
    neck_angle = angles.get("neck_angle", 0)

    score = 100
    feedback = []

    if shoulder_angle < 160:
        score -= (160 - shoulder_angle) * 0.5
        feedback.append("⚠️ Shoulders tilted forward — possible slouching.")
    if neck_angle < 150:
        score -= (150 - neck_angle) * 0.7
        feedback.append("⚠️ Neck bent — possible strain risk.")

    score = np.clip(score, 0, 100)
    risk = "Low" if score > 85 else "Medium" if score > 60 else "High"

    return {"score": float(score), "risk_level": risk, "feedback": feedback or ["✅ Good posture."]}


def draw_feedback(frame, angles, risk_info):
    """Overlay results on video frame."""
    color = (0, 255, 0) if risk_info["risk_level"] == "Low" else \
            (0, 255, 255) if risk_info["risk_level"] == "Medium" else (0, 0, 255)

    cv2.putText(frame, f"Risk: {risk_info['risk_level']}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
    cv2.putText(frame, f"Score: {risk_info['score']:.1f}", (10, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 1)

    y = 100
    for line in risk_info["feedback"]:
        cv2.putText(frame, line, (10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 1)
        y += 25

    return frame


In [ ]:
# Upload video file
print("📤 Upload your .mp4 video for analysis:")
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# Initialize video capture
cap = cv2.VideoCapture(filename)
fps = cap.get(cv2.CAP_PROP_FPS)
w, h = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter("output.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"🎞 Processing {frame_count} frames...")

for _ in tqdm(range(frame_count)):
    ret, frame = cap.read()
    if not ret:
        break

    annotated, angles = detect_pose_angles(frame, pose)
    if angles:
        risk = assess_posture_risk(angles)
        annotated = draw_feedback(annotated, angles, risk)

    out.write(annotated)

cap.release()
out.release()

print("✅ Processing complete! Output saved as 'output.mp4'")


In [ ]:
# Display video inline in Colab
mp4 = open('output.mp4', 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width=640 controls>
    <source src="{data_url}" type="video/mp4">
</video>
""")

# Download the video
files.download('output.mp4')
